<h2>Adding imports</h2>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from timeit import timeit
from IPython.display import display
import matplotlib
# matplotlib.use("pgf")
from matplotlib import pyplot as plt
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from collections import Counter
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import cohen_kappa_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from joblib import dump, load
from imblearn.under_sampling import RandomUnderSampler
import shap
import seaborn as sns

Open the dataset<br>
The compile_dataset() function reads the dataset from file.<br>
The original datasets contain these features:<br>
timestamp/t (absolute), ID, DLC (some don't have this), data field 0, ..., data field 7, Label.<br>
While compiling the dataset from the loaded files, new features are created.<br>
From the timestamp, "dt" is created, and from timestamp and ID, "dt_ID" is created.<br>
dt is the time between each CAN frame. dt_ID is the time between each CAN frame with the same ID.<br>
From data field 0, ..., data field 7, "ones" is created, and from ID, "ones_ID" is created.<br>
ones is the number of ones counted in the binary payload (combined data fields) for each CAN frame,<br>
and ones_ID is the number of ones counted in the ID for each CAN frame.<br>
Another feature, "type" is also added, which denotes the type of attack that a certain attack is.<br>
For example, flooding (flood), and fuzzing (fuzz).

In [ ]:
from compiledataset import load_dataset, compile_dataset

PATH = "/home/hampus/miun/master_thesis/Datasets"

datasets = {}

# dataset: pd.DataFrame = load_dataset(PATH + "/ORNL", "data_a.csv")
# dataset["remarks"] = "No DLC available"
# datasets["ROAD"] = dataset.to_dict("records")

dataset: pd.DataFrame = load_dataset(PATH + "/Survival", "data.csv") #, "FreeDrivingData_20180112_KIA")
dataset["remarks"] = "-"
datasets["Survival"] = dataset.to_dict("records")

# dataset: pd.DataFrame = load_dataset(PATH + "/Hisingen", "data.csv")
# dataset["remarks"] = "-"
# datasets["Hisingen"] = dataset.to_dict("records")


df = compile_dataset(datasets)

dataset = None # Release memory, as it isn't used for now
datasets = None

display(df[df["Label"] == 1][["dt", "ID", "d0", "d1", "d2", "d3", "d4", "d5", "d6", "d7"]])
print(df["dt"].mean())

Plots of the delta time of the datasets

In [ ]:
from matplotlib.ticker import StrMethodFormatter

# df["dt"] *= 1000
# df["t"] -= 6e7
# df = df.sample(frac=0.3, random_state=0)


normal = df.loc[(df["Label"] == 0), "dt_ID"]
normal_t = df.loc[(df["Label"] == 0), "t"]
attack = df.loc[(df["Label"] == 1), "dt_ID"]
attack_t = df.loc[(df["Label"] == 1), "t"]
# print(len(attack))

# normal = df.loc[(df["name"] == "Malfunction_dataset_SONATA") & (df["Label"] == 0), "dcs"]
# normal_t = df.loc[(df["name"] == "Malfunction_dataset_SONATA") & (df["Label"] == 0), "t"]
# attack = df.loc[(df["name"] == "Malfunction_dataset_SONATA") & (df["Label"] == 1), "dcs"]
# attack_t = df.loc[(df["name"] == "Malfunction_dataset_SONATA") & (df["Label"] == 1), "t"]
plt.figure(figsize=(9.6, 1.4), dpi=200)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.1f}'))
plt.scatter(x=normal_t, y=normal*1000, s=0.5, label="Normal")
plt.scatter(x=attack_t, y=attack*1000, s=0.5, c="red", label="Attack")

# plt.ylim(0, 10)

plt.xlabel("Elapsed time (s)")
plt.ylabel("$\Delta$ t (µs)")
plt.legend(markerscale=8, loc="upper right")
# plt.title("Chevrolet Spark")

plt.show()

# plt.savefig("ROAD_correlated_signal_attack_1_dt.png")

Look at dt_IDs

In [ ]:
plt.hist(
    x=[
        df.loc[df["Label"] == 0, "dt"],
        df.loc[df["type"] == "masq", "dt"],
        df.loc[df["type"] == "fuzz", "dt"],
        df.loc[df["type"] == "fabr", "dt"]
    ],
    bins=100,
    range=(0, 0.01),
    color=["tab:blue", "tab:red", "tab:green", "tab:purple"],
    log=True,
    label=["Normal", "masq", "fuzz", "fabr"], histtype="step"
)
plt.legend()
plt.xlabel("Time between packets with the same ID\n(Divided into 100 steps)")
plt.ylabel("Number of CAN frames")
# plt.hist(df.loc[df["Label"] == 1, "dt_ID"], bins=30, range=(0, 50), color="tab:red", log=True, rwidth=0.5)
plt.show()

Measure the occurence of IDs in the normal data

In [ ]:
types, types_count = np.unique(df.loc[df["Label"] == 0, "ID"], return_counts=True)
t_dt = []
for t in types:
    t_dt.append(df.loc[(df["ID"] == t) & (df["Label"] == 0), "dt_ID"].mean())
plt.bar(x=types, height=t_dt)#, log=True)
plt.show()
print(df["dt_ID"].max())

Drop a few columns<br>
The data fields are dropped because there are different standards for each car manufacturers<br>
on how to use the bytes in the payload of the CAN frame.<br>
The ID is dropped for the same reason as the data fields; manufacturers use different high level protocols<br>
to encode the ID field in their vehicles. Also, some datasets obfuscate their data.
DLC is dropped because it does not exist in all datasets.<br>
t is dropped because the absolute timestamps will not give much information, because<br>
data dathered on different times will vary a lot. The delta time (dt) is better in this regard.<br>

In [ ]:
df.drop(columns=["d0", "d1", "d2", "d3", "d4", "d5", "d6", "d7", "ID", "DLC", "t"], inplace=True, errors="ignore")

display(df)

Perform a Chi Squared test to see if there are any features that aren't dependent on the classification label, "Label".

In [ ]:
# Use a dataframe without "type", as it is categorical and will not be used to train later
# Drop dt and dt_ID as they are not categorical
X_chi = df.drop(columns=["dt" ,"dt_ID", "type", "dataset", "Label"])
y_chi = df["Label"]
chi_scores = chi2(X_chi, y_chi)
p_values = chi_scores[1]
print(p_values)
plt.bar(x=X_chi.columns, height=p_values)
plt.show()
X_chi = None # Release memory
y_chi = None

I believe this means there is not sufficient evidence to disprove the null hypothesis (i.e. that features are independent of class label).

We can also check the correlation between the features:

In [ ]:
# Compute the correlation matrix
corr = df.drop(columns=["dataset", "type"]).corr()

# Drop first row and last column that don't provide information
corr.drop(index=corr.index[0], inplace=True)
corr.drop(columns=corr.columns[-1], inplace=True)

# Generate a mask for the upper triangle but not the diagonal
mask = np.triu(np.ones_like(corr, dtype=bool))
np.fill_diagonal(mask, False)

# Draw the correlation heatmap with the mask
def tostr(num):
    if isinstance(num, str): return num
    if num < 0: return str(num)[:5]
    return str(num)[:4]
def remove_nocorr(corr):
    annot = corr.copy()
    annot.where(np.abs(annot) > 0.1, " ", inplace=True)
    annot = annot.applymap(tostr)
    return annot
annots = remove_nocorr(corr)

sns.heatmap(corr, mask=mask, vmin=-1, vmax=1, center=0, annot=annots, annot_kws={"fontsize": 8}, fmt="s", square=True)
plt.show()

We will now see if the dataset is imbalanced, and also how we should stratify when creating the training and test sets.<br>
Stratification makes sure that there is the same proportion of a class in both the training and test sets.<br>
If the whole dataset contains 1% of one class and 99% of another, then the training and test sets will also have that proportion.<br>
We begin by looking at the proportion of Normal vs Attack instances:

In [ ]:
plt.bar(x=["Normal", "Attack"], height=np.bincount(df["Label"]), color=["tab:blue", "tab:red"])
plt.show()

Next, we look at the proportion of the different types of attacks:

In [ ]:
types, types_count = np.unique(df.loc[df["type"] != "none", "type"], return_counts=True)
plt.bar(x=types, height=types_count, color=["tab:green", "tab:red", "tab:orange", "tab:blue"])
plt.show()

It can be seen that there are different amounts of attack types in the dataset.<br>
We want the proportion to be the same after splitting into training and testing data.

Visualize the distribution of attack instances by ones counted.

In [ ]:
# d_temp = pd.concat([X_train, y_train], axis="columns")
size = len(df.loc[df["Label"] == 1].index)
ones_prob = [[], [], []]
# zeros_prob = []
for i in range(65):
    # zeros_prob.append(len(d_temp.loc[(d_temp["Label"] == 0) & (d_temp["ones"] == i)].index) / size)

    prob_flood = len(df.loc[(df["Label"] == 1) & (df["ones"] == i) & (df["type"] == "masq")].index) / size
    prob_fuzz = len(df.loc[(df["Label"] == 1) & (df["ones"] == i) & (df["type"] == "fuzz")].index) / size
    prob_fabr = len(df.loc[(df["Label"] == 1) & (df["ones"] == i) & (df["type"] == "fabr")].index) / size
    ones_prob[0].append(prob_flood)
    ones_prob[1].append(prob_fuzz)
    ones_prob[2].append(prob_fabr)


# zeros_prob = pd.Series(zeros_prob)
ones_prob = pd.Series(ones_prob)
# sns.lineplot(data=ones_prob)

# plt.bar(x=range(0, 65), height=zeros_prob)
plt.bar(x=range(0, 65), height=ones_prob[0], color="tab:red")
plt.bar(x=range(0, 65), height=ones_prob[1], color="tab:green")
plt.bar(x=range(0, 65), height=ones_prob[2], color="tab:orange")
plt.legend(labels=["Flooding", "Fuzzing", "Fabrication"])
plt.xlabel("Number of ones counted")
plt.ylabel("% of attack observations (Label = 1)")
plt.show()

It shows that in the flooding attack, there is never any ones, and it makes up almost 50% of all attacks.<br>
It also shows that the fuzzing attack is randomly generating payloads (as is stated in the paper as well), and since there is a higher chance that a payload will have a one count of 32 (by binomial distribution) the distribution looks the way it does.<br>
It shows that the fabrication attacks have 20 ones, 29 ones or randomly chose (it is the same as fuzzing, just smaller so is not seen well on the graph, but it is seen if plotted separately.).<br>
It shows that 50% of the attacks contain no ones in the payload, so you will catch many attacks going just on that!
Moreover, these attacks have clear patterns by how many ones they contain, as they are either a Specific value (0, 20 or 29).. or... binomially distributed.

Create the training and test data (with stratification on the type of attack)

In [ ]:
X_train = df.drop(columns="type")
y_train = df["type"]

df = None # Release memory

# Split dataset into training and test data, stratify by the type of attack
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=0, shuffle=True, stratify=y_train)

# # Under-sample on type
# rus = RandomUnderSampler(random_state=0)
# X_train, y_train = rus.fit_resample(X_train, y_train)

# Use feature "Label" as classification label
d_temp: pd.DataFrame = pd.concat([X_train, y_train], axis="columns")
X_train, y_train = d_temp.drop(columns="Label"), d_temp["Label"]
d_temp: pd.DataFrame = pd.concat([X_test, y_test], axis="columns")
X_test, y_test = d_temp.drop(columns="Label"), d_temp["Label"]
d_temp = None # Release memory

Now, we can look at the proportions again (in the Training data):

In [ ]:
types, types_count = np.unique(X_train.loc[X_train["type"] != "none", "type"], return_counts=True)
plt.bar(x=types, height=types_count, color=["tab:orange", "tab:red", "tab:green", "tab:blue"])
plt.show()

Under-sample the training set by the majority class (no attack, 0). The test set can be left imbalanced, as real world data can also be imbalanced.

In [ ]:
rus = RandomUnderSampler(random_state=0)
X_train, y_train = rus.fit_resample(X_train, y_train)
bintr = np.bincount(y_train)
binte = np.bincount(y_test)
print(f"Labels\t\tTraining\tTesting\nNormal\t\t{bintr[0]}\t\t{binte[0]}\nAttack\t\t{bintr[1]}\t\t{binte[1]}")